In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [74]:
# Set up the Selenium WebDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode (optional)
driver = webdriver.Chrome(service=service, options=options)

In [75]:
# Navigate to the website
url = 'https://www.goodschools.com.au/compare-schools/search/in-victoria'
driver.get(url)
time.sleep(5)  # Wait for the page to load completely

In [76]:
# Initialize data storage
schools_data = []

In [77]:
# # 抓取学校数据的循环
# while len(schools_data) < 50:
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
    
#     # 获取每个学校的卡片
#     school_cards = soup.find_all('div', class_='media-body')

#     for card in school_cards:
#         try:
#             # 抓取学校名称
#             name = card.find('h5', class_='mb-0 font-weight-bold').text.strip()

#             # 抓取学校地址
#             address = card.find('p', class_='primary-site').text.strip()

#             # 抓取 sector 信息
#             sector = None
#             sector_divs = card.find_all('div', class_='col-md-3 col-6 small')
#             for div in sector_divs:
#                 if "Sector" in div.text:
#                     sector = div.find_next_sibling('div').text.strip()
#                     break

#             # 将抓取到的数据加入列表
#             schools_data.append({
#                 'Name': name,
#                 'Address': address,
#                 'Sector': sector,
#             })

#             if len(schools_data) >= 50:
#                 break

#         except AttributeError:
#             continue  # 如果某个字段缺失则跳过该项

#     # 点击 "Next" 按钮进入下一页
#     try:
#         next_button = driver.find_element_by_link_text('Next')
#         next_button.click()
#         time.sleep(5)  # 等待下一页加载
#     except:
#         break  # 如果没有更多页面则退出循环

# # 关闭 WebDriver
# driver.quit()

In [78]:
# sector 在外面抓，其他在里面抓，next有问题
while len(schools_data) < 50:
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 获取每个学校的卡片
    school_cards = soup.find_all('div', class_='media-body')
    
    # 抓取 sector 信息 no data
    sector = None
    sector_divs = card.find_all('div', class_='col-md-3 col-6 small')
    for div in sector_divs:
        if "Sector" in div.text:
            sector = div.find_next_sibling('div').text.strip()
            break

    for card in school_cards:
        try:
            # 抓取学校名称
            name = card.find('h5', class_='mb-0 font-weight-bold').text.strip()

            # 获取学校详情页链接
            school_link = card.find('a', href=True)['href']
            # school_url = f"https://www.goodschools.com.au{school_link}"
            # print(school_url)  # 输出 URL 检查格式是否正确

            # 进入学校详情页面
            driver.get(school_link)
            time.sleep(3)  # 等待详情页面加载
            school_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 抓取学校的 postcode 和 geolocation（假设 geolocation 可用）
            try:
                postcode = school_soup.find('span', class_='postal-code').text.strip()
                geolocation = school_soup.find('span', class_='geo-coordinates').text.strip()
            except AttributeError:
                postcode = None
                geolocation = None

            # 抓取 academic results
            try:
                academic_results = school_soup.find('div', class_='academic-results').text.strip()
            except AttributeError:
                academic_results = "N/A"

            # 将抓取到的数据加入列表
            schools_data.append({
                'Name': name,
                'Postcode': postcode,
                'Geolocation': geolocation,
                'Sector': sector,
                'Academic Results': academic_results
            })

            if len(schools_data) >= 50:
                break

        except AttributeError:
            continue  # 如果某个字段缺失则跳过该项

    # 返回到学校列表页面
    driver.get(url)
    time.sleep(5)  # 等待页面重新加载

    # 点击 "Next" 按钮进入下一页
    try:
        next_button = driver.find_element(By.LINK_TEXT, 'Next')
        next_button.click()
        time.sleep(5)  # 等待下一页加载
    except:
        break  # 如果没有更多页面则退出循环

# 关闭 WebDriver
driver.quit()

In [79]:
# Save data to CSV
df = pd.DataFrame(schools_data)
print(schools_data) 

df.to_csv('victoria_secondary_schools.csv', index=False)

print('Data extraction completed. CSV file has been saved.')


[{'Name': 'Box Hill Senior Secondary College', 'Address': 'Mont Albert North VIC, Melbourne Eastern Suburbs', 'Sector': None}, {'Name': 'Brighton Grammar School', 'Address': 'Brighton VIC, Melbourne Southern Suburbs & Western Port', 'Sector': None}, {'Name': 'Carey Baptist Grammar School', 'Address': 'Kew VIC, Melbourne Eastern Suburbs', 'Sector': None}, {'Name': 'Centre for Higher Education Studies', 'Address': 'South Yarra', 'Sector': None}, {'Name': 'Genazzano FCJ College', 'Address': 'KEW VIC, Melbourne Eastern Suburbs', 'Sector': None}, {'Name': 'Heritage College', 'Address': 'Narre Warren South VIC, Melbourne Southern Suburbs & Western Port', 'Sector': None}, {'Name': "St Catherine's School", 'Address': 'Toorak VIC, Melbourne Southern Suburbs & Western Port', 'Sector': None}, {'Name': "St Columba's College", 'Address': 'Essendon VIC, Melbourne Western Suburbs', 'Sector': None}, {'Name': "St Kevin's College", 'Address': 'Toorak VIC, Melbourne Southern Suburbs & Western Port', 'Sec